In [ ]:
!pip install biopython scikit-learn --quiet
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from random import shuffle
def load_sequences(file_paths, window=150, stride=30):
    sequences, labels = [], []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append((frag, label))
    shuffle(sequences)
    fragments, labels = zip(*sequences)
    return list(fragments), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    'Zika': '/content/drive/MyDrive/sagar/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/sagar/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/sagar/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/sagar/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/sagar/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/sagar/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=30)
print(f" Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=500)
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
centroid_model = NearestCentroidClassifier(n_estimators=300, max_depth=30, random_state=42)
centroid_model.fit(X_train_scaled, y_train)
y_pred_centroid = centroid_model.predict(X_test_scaled)
acc_centroid = accuracy_score(y_test, y_pred_centroid)
print(f"\n Nearest Centroid Accuracy: {acc_centroid * 100:.2f}%")
print("\n Nearest Centroid Classification Report:\n")
print(classification_report(y_test, y_pred_centroid))

 Augmented sequences: 16000

 Nearest Centroid Accuracy: 97.50%

 Nearest Centroid Classification Report:

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      1373
           1       1.00      0.94      0.97       699
           2       1.00      1.00      1.00       236
           3       0.96      0.95      0.96       376
           4       1.00      0.99      0.99       412
           5       1.00      0.85      0.92       101
           6       1.00      1.00      1.00         3

    accuracy                           0.97      3200
   macro avg       0.99      0.96      0.97      3200
weighted avg       0.98      0.97      0.97      3200

